# Descrição

*Código base para classificação de imagens usando redes convolucionais
*Bibliotecas
*Carregar Bases de dados
*Construir Modelos
*Treinamento
*Avaliação
*Hold out
Validação Cruzada

# Bibliotecas

In [10]:
import os
import zipfile
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from albumentations.core.composition import OneOf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split

# Carregar Bases dados

In [2]:
#Conexão com Colab
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)
from google.colab import files

Mounted at /content/gdrive


Carregar Base GAMMA

In [3]:
#Descompactar arquivo e salvar imagens (X) e CSV (LABELS)
path_to_dataset = '/content/gdrive/MyDrive/Doutorado/Bases/GAMMA_training_data/multi-modality_images/dataset_fundus/dataset_fundus.zip'

import zipfile

def descompactar_arquivo(zip_path, destino):
    """Descompacta um arquivo .zip e salva em uma pasta específica."""
    if not os.path.exists(destino):
        os.makedirs(destino)  # Cria a pasta de destino se não existir

    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(destino)
        print(f"Arquivos extraídos para: {destino}")

# Exemplo de uso
zip_path = path_to_dataset  # Substitua pelo caminho do seu arquivo .zip
destino = "/content/Dataset GAMMA 2"  # Substitua pelo caminho onde deseja salvar os arquivos

descompactar_arquivo(zip_path, destino)


Arquivos extraídos para: /content/Dataset GAMMA 2


In [6]:
def carregar_imagens_e_labels(dataset_path):
    """Carrega imagens da pasta e os labels do arquivo train.csv"""
    #IMG_SIZE = (128, 128)
    # Caminho do CSV com os labels
    csv_path = os.path.join(dataset_path, "train.csv")

    # Lendo os labels do CSV
    df = pd.read_csv(csv_path)

    # Criando listas para armazenar imagens e labels
    X = []  # Lista de imagens (arrays)
    y = []  # Lista de rótulos (classes)

    # Percorrendo cada linha do CSV
    for _, row in df.iterrows():
        #img_name = row.iloc[0]  # Nome do arquivo da imagem (ex: "0001.jpg")
        img_name = str(row.iloc[0]).zfill(4) + ".jpg"
        label = row.iloc[1:].values  # Labels (non, early, mid_advanced)

        # Determinando a classe (índice do label 1)
        class_id = np.argmax(label)  # Ex: [1, 0, 0] → 0 | [0, 0, 1] → 2

        # Caminho completo da imagem
        img_path = os.path.join(dataset_path, img_name)
        #print(img_path)

        # Verifica se a imagem existe antes de carregar
        if os.path.exists(img_path):
            # Carrega a imagem em escala de cinza (ou use cv2.IMREAD_COLOR para RGB)
            #img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            img = cv2.imread(img_path, cv2.IMREAD_COLOR)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

            # Redimensiona para um tamanho fixo (opcional, ex: 128x128)
            img = cv2.resize(img, IMG_SIZE)

            # Normaliza os pixels (0 a 1)
            img = img.astype(np.float32) / 255.0

            # Adiciona à lista
            X.append(img)
            y.append(class_id)
        else:
            print(f"⚠️ Aviso: Imagem {img_name} não encontrada.")

    # Convertendo listas para arrays NumPy
    #X = np.array(X).reshape(-1, 128, 128, 1)  # Mantém formato para CNNs
    X = np.array(X)
    y = np.array(y)

    print(f"✅ {len(X)} imagens carregadas com sucesso!")
    return X, y

# Caminho para a pasta do dataset
dataset_path = "/content/Dataset GAMMA 2"
IMG_SIZE = (128, 128)
# Chamando a função para carregar imagens e labels
X, y = carregar_imagens_e_labels(dataset_path)

# Verificando as dimensões
print(f"Formato de X: {X.shape}, Formato de y: {y.shape}")


✅ 100 imagens carregadas com sucesso!
Formato de X: (100, 128, 128, 3), Formato de y: (100,)


# Construir Modelos

In [9]:
# Definição da CNN
def criar_modelo(input_shape, num_classes):
    model = keras.Sequential([
        # 🔹 Bloco convolucional 1
        layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=input_shape),
        layers.MaxPooling2D((2, 2)),

        # 🔹 Bloco convolucional 2
        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D((2, 2)),

        # 🔹 Achatar para camada totalmente conectada
        layers.Flatten(),

        # 🔹 Camada densa 1
        layers.Dense(128, activation='relu'),

        # 🔹 Camada densa final (softmax para classificação)
        layers.Dense(num_classes, activation='softmax')
    ])

    # Compilar o modelo
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    return model

# 🔹 Definições de entrada
#input_shape = ((IMG_SIZE), 3)  # Exemplo: imagens coloridas 128x128
input_shape = (IMG_SIZE[0], IMG_SIZE[1], 3)# Exemplo: imagens coloridas 128x128
num_classes = len(np.unique(y))  # Exemplo: 3 categorias

# Criar o modelo
modelo = criar_modelo(input_shape, num_classes)

# Resumo da arquitetura
modelo.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_4 (Conv2D)                    │ (None, 128, 128, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_4 (MaxPooling2D)       │ (None, 64, 64, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_5 (Conv2D)                    │ (None, 64, 64, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_5 (MaxPooling2D)       │ (None, 32, 32, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_2 (Flatten)                  │ (None, 65536)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 128)                 │       8,388,736 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 3)                   │             387 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 8,408,515 (32.08 MB)

 Trainable params: 8,408,515 (32.08 MB)

 Non-trainable params: 0 (0.00 B)

# Treino

In [12]:
def treinar_holdout(model, X, y, test_size, epochs, batch_size):
    """
    Treina um modelo CNN usando Hold-Out.

    Parâmetros:
        model: Modelo Keras a ser treinado.
        X: Dados de entrada (imagens).
        y: Rótulos correspondentes.
        test_size: Proporção dos dados para validação (default: 20%).
        epochs: Número de épocas de treinamento.
        batch_size: Tamanho do lote para o treinamento.

    Retorna:
        O histórico do treinamento.
    """
    # 🔹 Dividir os dados em treino e validação
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=test_size, random_state=42)

    # 🔹 Treinar o modelo
    history = model.fit(
        X_train, y_train,
        epochs=epochs,
        batch_size=batch_size,
        validation_data=(X_val, y_val)
    )

    return history

# Exemplo de uso
# Supondo que X e y sejam os dados carregados corretamente
# X.shape = (n_amostras, largura, altura, canais) | y.shape = (n_amostras,)
batch = 5
epochs = 30
test_size = 0.1
modelo = criar_modelo(input_shape=input_shape, num_classes=num_classes)
history = treinar_holdout(modelo, X, y, test_size, epochs, batch)

Epoch 1/30
18/18 ━━━━━━━━━━━━━━━━━━━━ 3s 37ms/step - accuracy: 0.5080 - loss: 1.1188 - val_accuracy: 0.8000 - val_loss: 0.9195
Epoch 2/30
18/18 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 0.6002 - loss: 0.8757 - val_accuracy: 0.7000 - val_loss: 1.3461
Epoch 3/30
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.6250 - loss: 0.7278 - val_accuracy: 0.7000 - val_loss: 0.9705
Epoch 4/30
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.7050 - loss: 0.6483 - val_accuracy: 0.6000 - val_loss: 0.8288
Epoch 5/30
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.7774 - loss: 0.5902 - val_accuracy: 0.7000 - val_loss: 1.1689
Epoch 6/30
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.7996 - loss: 0.5041 - val_accuracy: 0.7000 - val_loss: 1.1055
Epoch 7/30
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.8006 - loss: 0.4573 - val_accuracy: 0.7000 - val_loss: 1.5509
Epoch 8/30
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.7698 - loss: 0.5838 - val_accuracy: 0.7000 - v